# Common Variables.

## Average Calculations

In [1]:
import random

small_list = [random.randint(1, 100) for _ in range(50)]
medium_list = [random.randint(1, 100) for _ in range(5000)]
large_list = [random.randint(1, 100) for _ in range(500000)]
extra_large_list = [random.randint(1, 100) for _ in range(5000000)]

# PySpark Benchmark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
import time

In [3]:
#192.168.122.217    spark-master

MASTER_IP = "spark-master"

spark = (
    SparkSession.builder.appName("Thoth Benchmark")
    .master(f"spark://{MASTER_IP}:7077")
    # .config("spark.executor.memory", "1g")
    # .config("spark.driver.memory", "1g")
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/30 16:50:01 WARN Utils: Your hostname, akm, resolves to a loopback address: 127.0.1.1; using 192.168.1.8 instead (on interface wlp0s20f3)
25/08/30 16:50:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/30 16:50:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
small_df = spark.createDataFrame(small_list, "int").toDF("value")
medium_df = spark.createDataFrame(medium_list, "int").toDF("value")
large_df = spark.createDataFrame(large_list, "int").toDF("value")
extra_large_list_df = spark.createDataFrame(extra_large_list, "int").toDF("value")

start_time = time.time()
small_df.agg(avg("value")).show()
end_time = time.time()
py_spark_small_list_time = end_time - start_time
print(f"\nSpark Time taken to process small DataFrame: {py_spark_small_list_time} seconds")

start_time = time.time()
medium_df.agg(avg("value")).show()
end_time = time.time()
py_spark_medium_list_time = end_time - start_time
print(f"\nSpark Time taken to process medium DataFrame: {py_spark_medium_list_time} seconds")

start_time = time.time()
large_df.agg(avg("value")).show()
end_time = time.time()
py_spark_large_list_time = end_time - start_time
print(f"\nSpark Time taken to process large DataFrame: {py_spark_large_list_time} seconds")

start_time = time.time()
extra_large_list_df.agg(avg("value")).show()
end_time = time.time()
py_spark_extra_large_list_time = end_time - start_time
print(f"\nSpark Time taken to process extra large DataFrame: {py_spark_extra_large_list_time} seconds")

+----------+
|avg(value)|
+----------+
|     57.36|
+----------+


Time taken to process small DataFrame: 2.987133741378784 seconds
+----------+
|avg(value)|
+----------+
|   50.7672|
+----------+


Time taken to process medium DataFrame: 0.20919013023376465 seconds
+----------+
|avg(value)|
+----------+
|   50.4502|
+----------+


Time taken to process large DataFrame: 0.40839648246765137 seconds


25/08/30 16:50:13 WARN TaskSetManager: Stage 9 contains a task of very large size (6561 KiB). The maximum recommended task size is 1000 KiB.


+----------+
|avg(value)|
+----------+
|50.5175776|
+----------+


Time taken to process extra large DataFrame: 1.2565114498138428 seconds


25/08/30 16:50:24 WARN StandaloneAppClient$ClientEndpoint: Connection to 192.168.122.217:7077 failed; waiting for master to reconnect...
25/08/30 16:50:24 WARN StandaloneSchedulerBackend: Disconnected from Spark cluster! Waiting for reconnection...
25/08/30 16:50:29 ERROR TaskSchedulerImpl: Lost executor 0 on 0.0.0.0: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.


# Thoth BenchMark

In [5]:
from py_thoth.operations.vector import list_average
from py_thoth.settings.connections import change_remote_address
remote_addresses = ["192.168.122.217:50051", "192.168.122.215:50051"]
# remote_addresses = ["localhost:50051", "localhost:50052"]
# remote_addresses = ["localhost:50051"]
change_remote_address(remote_addresses)


Remote address changed to: ['192.168.122.217:50051', '192.168.122.215:50051']


In [14]:
start_time = time.time()
print("Average Small List: ", list_average(small_list))
end_time = time.time()
thoth_small_list_time = end_time - start_time
print(f"\nThoth Small List Time taken: {thoth_small_list_time} seconds")

start_time = time.time()
print("Average Medium List: ", list_average(medium_list))
end_time = time.time()
thoth_medium_list_time = end_time - start_time
print(f"\nThoth Medium List Time taken: {thoth_medium_list_time} seconds")

start_time = time.time()
print("Average Large List: ", list_average(large_list))
end_time = time.time()
thoth_large_list_time = end_time - start_time
print(f"\nThoth Large List Time taken: {thoth_large_list_time} seconds")

start_time = time.time()
print("Average Extra Large List: ", list_average(extra_large_list))
end_time = time.time()
thoth_extra_large_list_time = end_time - start_time
print(f"\nThoth Extra Large Time taken: {thoth_extra_large_list_time} seconds")


Configurations remote address: ['192.168.122.217:50051', '192.168.122.215:50051']
Average Small List:  57.36

Thoth Small List Time taken: 0.040738821029663086 seconds
Configurations remote address: ['192.168.122.217:50051', '192.168.122.215:50051']
Average Medium List:  50.7672

Thoth Medium List Time taken: 0.04539155960083008 seconds
Configurations remote address: ['192.168.122.217:50051', '192.168.122.215:50051']


KeyboardInterrupt: 